## [Sec1] create Sparse Matrix with pd get_dummies (failure)

fail due to >pd.get_dummies is extremely unefficientcy

In [1]:
import pandas as pd
import numpy as np
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

In [2]:
features=X_test.columns.values[2:]

In [3]:
features = X_test.columns.values[2:]
for column in features:
    print("handling col : ",column )
    categories = pd.concat([X_train[column],X_test[column]]).unique()
    X_train[column] = X_train[column].astype('category', categories=categories)
    X_test[column] = X_test[column].astype('category', categories=categories)

handling col :  C1
handling col :  banner_pos
handling col :  site_id
handling col :  site_domain
handling col :  site_category
handling col :  app_id
handling col :  app_domain
handling col :  app_category
handling col :  device_id
handling col :  device_ip
handling col :  device_model
handling col :  device_type
handling col :  device_conn_type
handling col :  C14
handling col :  C15
handling col :  C16
handling col :  C17
handling col :  C18
handling col :  C19
handling col :  C20
handling col :  C21


In [5]:
X_train.to_pickle("Train_cate")
X_test.to_pickle("Test_cate")

In [1]:
import pandas as pd

In [2]:
X_train=pd.read_pickle("Train_cate")
X_test=pd.read_pickle("Test_cate")

In [7]:
# split data into 5 folds
import numpy as np
def n_fold_split(df, k, seed=5566):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    fold_size = int(m/k)
    fold_list = []
    for i in range(k):
        if i!=(k-1):
            target_range=range(i*fold_size,(i+1)*fold_size)
        else:
            target_range=range(i*fold_size,m)
        current_df = df.iloc[perm[target_range]]
        fold_list.append(current_df)
    return fold_list

In [8]:
X_train_list = n_fold_split(X_train,5)
for i in X_train_list:
    print("batchsize:",len(i))

batchsize: 8085793
batchsize: 8085793
batchsize: 8085793
batchsize: 8085793
batchsize: 8085795


In [9]:
features=X_test.columns.values[2:]

In [10]:
features

array(['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category',
       'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip',
       'device_model', 'device_type', 'device_conn_type', 'C14', 'C15',
       'C16', 'C17', 'C18', 'C19', 'C20', 'C21'], dtype=object)

In [ ]:
test = pd.get_dummies(X_test, columns=features, sparse=True)
# extremely slowly and run-out memory (even with only Test dataset)

# [Sec2] manually build csr matrix, and fit xgb/SGD model
what we want -> a very large matrix with all category variable were translated to dummy

and sloving with logistic regression

## [Sec2-1]   load df which were built in FF ipynb,  and split it into 5 folds

In [1]:
import pandas as pd 
X_train = pd.read_pickle('train_df_hash')
X_test = pd.read_pickle('test_df_hash')

In [2]:
column2handle = X_train.columns.values[1:]

### split to 5 folds

In [3]:
# split data into 5 folds
import numpy as np
def n_fold_split(df, k, seed=5566):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    fold_size = int(m/k)
    fold_list = []
    for i in range(k):
        if i!=(k-1):
            target_range=range(i*fold_size,(i+1)*fold_size)
        else:
            target_range=range(i*fold_size,m)
        current_df = df.iloc[perm[target_range]]
        fold_list.append(current_df)
    return fold_list

In [4]:
X_train_list = n_fold_split(X_train,5)
for i in X_train_list:
    print("batchsize:",len(i))

batchsize: 8085793
batchsize: 8085793
batchsize: 8085793
batchsize: 8085793
batchsize: 8085795


###  [Sec 2-1-0] Test the format of sparse matrix
said dok_matrix/lil_matrix is good at constructing sparse matrices.

give a naive try to this two type and save it as csr_matrix

[REF](http://m.blog.csdn.net/pipisorry/article/details/41762945)

In [5]:
from scipy.sparse import dok_matrix

In [7]:
# test on dok_matrix
X_train_dok = dok_matrix((8085793, 10269200), dtype=np.bool)

In [9]:
%%time
for i in column2handle:
    target = X_train_list[0][i]
    X_train_dok[range(8085793),target.values]=1
X_train_dok = X_train_dok.tocsr()

CPU times: user 8min 51s, sys: 6.21 s, total: 8min 57s
Wall time: 9min 1s


In [10]:
import gc
gc.collect()

162

In [11]:
# test on lil_matrix
from scipy.sparse import lil_matrix

In [12]:
X_train_lil = lil_matrix((8085793, 10269200), dtype=np.bool)

In [13]:
%%time
for i in column2handle:
    target = X_train_list[0][i]
    X_train_lil[range(8085793),target.values]=1
X_train_lil = X_train_lil.tocsr()

CPU times: user 54.6 s, sys: 6.13 s, total: 1min
Wall time: 1min 1s


In [12]:
import gc
gc.collect()

7421

### [sec 2-1-1] filling hash values into sparse matrix 
### using lil_matrix is much more faster for assigning values

In [14]:
# save/load function for  csr_matrix
from scipy.sparse import csr_matrix
import numpy as np
def save_sparse_csr(filename, array):
    # note that .npz extension is added automatically
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)

def load_sparse_csr(filename):
    # here we need to add .npz extension manually
    loader = np.load(filename + '.npz')
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape'])

In [18]:
%%time
for j in range(5):
    X_train_lil = lil_matrix(( len(X_train_list[j]), 10269200), dtype=np.bool)
    gc.collect()
    for i in column2handle:
        print(i)
        target = X_train_list[j][i]
        X_train_lil[range(len(X_train_list[j])),target.values]=1
        
    X_train_lil = X_train_lil.tocsr()
    save_sparse_csr('data/X_train_csr_'+str(j), X_train_lil)
    np.save('data/X_train_csr_click_'+str(j), X_train_list[j].click.values)
    gc.collect()

C1_hash
banner_pos_hash
site_id_hash
site_domain_hash
site_category_hash
app_id_hash
app_domain_hash
app_category_hash
device_id_hash
device_ip_hash
device_model_hash
device_type_hash
device_conn_type_hash
C14_hash
C15_hash
C16_hash
C17_hash
C18_hash
C19_hash
C20_hash
C21_hash
weekday_hash
hour__hash
C1_hash
banner_pos_hash
site_id_hash
site_domain_hash
site_category_hash
app_id_hash
app_domain_hash
app_category_hash
device_id_hash
device_ip_hash
device_model_hash
device_type_hash
device_conn_type_hash
C14_hash
C15_hash
C16_hash
C17_hash
C18_hash
C19_hash
C20_hash
C21_hash
weekday_hash
hour__hash
C1_hash
banner_pos_hash
site_id_hash
site_domain_hash
site_category_hash
app_id_hash
app_domain_hash
app_category_hash
device_id_hash
device_ip_hash
device_model_hash
device_type_hash
device_conn_type_hash
C14_hash
C15_hash
C16_hash
C17_hash
C18_hash
C19_hash
C20_hash
C21_hash
weekday_hash
hour__hash
C1_hash
banner_pos_hash
site_id_hash
site_domain_hash
site_category_hash
app_id_hash
app_domai

In [ ]:
# X_test 
X_test_lil = lil_matrix(( len(X_test), 10269200), dtype=np.bool)
for i in column2handle:
    print(i)
    target = X_test[i]
    X_test_lil[range(len(X_test)),target.values]=1

X_test_lil = X_test_lil.tocsr()
save_sparse_csr('data/X_test_csr_'+str(j), X_test_lil)
gc.collect()

# [Sec3] build SGD , XGB models

In [8]:
# save csr_matrix
from scipy.sparse import csr_matrix
import numpy as np

def load_sparse_csr(filename):
    # here we need to add .npz extension manually
    loader = np.load(filename + '.npz')
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape'])

In [13]:
X_train_csr = []
Y_train = []
for i in range(5):
    X_train_csr.append(load_sparse_csr('data/X_train_csr_'+str(i)))
    Y_train.append(np.load('data/X_train_csr_click_'+str(i)+'.npy'))

## [Sec 3-0] Familiar with scikit-learn SGD Classifier

In [174]:
from sklearn.linear_model import SGDClassifier
clf=SGDClassifier(loss="log",  eta0=1e-5, n_iter=150,
                  alpha=1e-5 ,penalty="l2",verbose=1,
                  learning_rate="constant") 

#n_jobs=-1 : number of cpu used, this option is used in one-verse-all

In [193]:
%%time 
# with using paritial fit 
clf.partial_fit(X_train_csr_0, Y_train_0)

-- Epoch 1
Norm: 10.74, NNZs: 3551169, Bias: -0.006885, T: 8085793, Avg. loss: 0.401061
Total training time: 3.18 seconds.
CPU times: user 4.63 s, sys: 128 ms, total: 4.76 s
Wall time: 4.2 s


SGDClassifier(alpha=1e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=1e-05, fit_intercept=True, l1_ratio=0.15,
       learning_rate='constant', loss='log', n_iter=150, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=1, warm_start=False)

In [176]:
clf.score(X_train_csr_0,Y_train_0)

0.83404027285882787

In [177]:
y_train_0_hat=clf.predict_log_proba(X_train_csr_0)

In [178]:
sum(np.exp(y_train_0_hat[0])) # really is log(p_0), log(p_1)

1.0

In [180]:
# # i-th data
# likelihood is calculated with pi^(yi)*(1-pi)^(1-yi)
# log-likelihood is yi*log(pi)+(1-yi)*log(1-pi)
# loss is - sum(log-likelihood)
# the log-loss function
np.mean(y_train_0_hat[range(len(Y_train_0)),Y_train_0])*(-1)

0.40109270011955145

In [194]:
# predict on validation set 1~4
y_train_1_hat=clf.predict_log_proba(X_train_csr[1])
np.mean(y_train_1_hat[range(len(Y_train_0)),Y_train[1]])*(-1)

0.40085742973102617

## [Sec 3 - 1]  Training with SGD, define evaluate, benchmark function

In [1]:
# load csr_matrix
from scipy.sparse import csr_matrix
import numpy as np

def load_sparse_csr(filename):
    # here we need to add .npz extension manually
    loader = np.load(filename + '.npz')
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape'])

In [2]:
# loading data from files
X_train_csr = []
Y_train = []
for i in range(5):
    X_train_csr.append(load_sparse_csr('data/X_train_csr_'+str(i)))
    Y_train.append(np.load('data/X_train_csr_click_'+str(i)+'.npy'))

In [3]:
def loss_evaluate(clf,X_test,Y_test):
    Y_hat=clf.predict_log_proba(X_test)
    return np.mean(Y_hat[range(len(Y_test)),Y_test])*(-1)

In [10]:
def benchmark(eta_=1e-5, alpha_=1e-5, fold_train=[0], early_stop=2):
    clf = SGDClassifier(loss="log",  eta0=eta_, 
                        alpha=alpha_ ,penalty="l2",
                        average=True,verbose=0)
          #learning_rate="constant") 
    classes = np.array([0,1])
    min_loss  = 100
    stop_count = 0
    best_epoch = 0
    for epoch in range(3000):
        for idx in fold_train:
            clf.partial_fit(X_train_csr[idx], Y_train[idx], 
                            classes=classes)
        if(epoch%10==0):
            print('_' * 80)
            
            print("Training epoch: ",epoch)
            train_err=[]
            for idx in fold_train:
                fold_loss = loss_evaluate(clf, X_train_csr[idx], Y_train[idx])
                train_err.append(fold_loss)
            print("** Training err = ",np.mean(train_err))
            
            print("Evaluate on Validation set:")
            testing_err = []
            for v in range(0,5):
                if v not in fold_train:
                    test_loss = loss_evaluate(clf, X_train_csr[v], Y_train[v])
                    testing_err.append(test_loss)
            w = np.mean(testing_err)
            print("** Validation err = ",w)
            
            if w <= min_loss:
                best_epoch = epoch
                min_loss = w
                best_epoch_train_error = np.mean(train_err)
            else:
                stop_count+=1
                if stop_count>=early_stop:
                    break
    print("best epoch is : ", best_epoch,", with loss : " ,min_loss)
    return (best_epoch, min_loss, best_epoch_train_error)
    

In [220]:
benchmark(eta_=5e-5, alpha_=0, fold_train=[0,1,2,3])

________________________________________________________________________________
Training epoch:  0
** Training err =  0.407077696484
** Evaluate on Validation set:
Validation err =  0.407167750835
________________________________________________________________________________
Training epoch:  10
** Training err =  0.399717436783
** Evaluate on Validation set:
Validation err =  0.399915028708
________________________________________________________________________________
Training epoch:  20
** Training err =  0.398233745033
** Evaluate on Validation set:
Validation err =  0.398529681716
________________________________________________________________________________
Training epoch:  30
** Training err =  0.397432463073
** Evaluate on Validation set:
Validation err =  0.397808687287
________________________________________________________________________________
Training epoch:  40
** Training err =  0.396803993206
** Evaluate on Validation set:
Validation err =  0.397264137772
______

(380, 0.39330078837013349)

In [ ]:
# (eta=5e-5, alpha=0), epoch:295 loss:0.3962 (train on fold0, test on fold1~4)
# (eta=5e-5, alpha=0), epoch:380 loss:0.3933 (train on fold0~3, test on fold4)

## [Sec 3-2] start training, will use ~15G memory, and one core()

In [6]:
from sklearn.linear_model import SGDClassifier

In [11]:
grid_result = dict()
grid_lr =[3e-5,1e-5,1e-6]
grid_alpha = [0]  #[0,1e-5,1e-6]
for lr in grid_lr:
    for alpha in grid_alpha:
        records = benchmark(eta_=lr, alpha_=alpha, fold_train=[0,1,2,3])
        grid_result[(lr,alpha)] = records
        

________________________________________________________________________________
Training epoch:  0
** Training err =  0.409330430133
Evaluate on Validation set:
** Validation err =  0.40936286427
________________________________________________________________________________
Training epoch:  10
** Training err =  0.401027338815
Evaluate on Validation set:
** Validation err =  0.401130390836
________________________________________________________________________________
Training epoch:  20
** Training err =  0.39942525163
Evaluate on Validation set:
** Validation err =  0.399595211851
________________________________________________________________________________
Training epoch:  30
** Training err =  0.398544778861
Evaluate on Validation set:
** Validation err =  0.398768659797
________________________________________________________________________________
Training epoch:  40
** Training err =  0.39792379286
Evaluate on Validation set:
** Validation err =  0.39820554007
__________

** Training err =  0.392384941526
Evaluate on Validation set:
** Validation err =  0.394101304949
________________________________________________________________________________
Training epoch:  420
** Training err =  0.392309671933
Evaluate on Validation set:
** Validation err =  0.394062597458
________________________________________________________________________________
Training epoch:  430
** Training err =  0.39222525617
Evaluate on Validation set:
** Validation err =  0.394013183844
________________________________________________________________________________
Training epoch:  440
** Training err =  0.392148051243
Evaluate on Validation set:
** Validation err =  0.393969606961
________________________________________________________________________________
Training epoch:  450
** Training err =  0.392085571461
Evaluate on Validation set:
** Validation err =  0.393939901726
________________________________________________________________________________
Training epoch:  460
*

** Training err =  0.389594837624
Evaluate on Validation set:
** Validation err =  0.392647391469
________________________________________________________________________________
Training epoch:  830
** Training err =  0.389534626176
Evaluate on Validation set:
** Validation err =  0.392615600565
________________________________________________________________________________
Training epoch:  840
** Training err =  0.389492744544
Evaluate on Validation set:
** Validation err =  0.39260124858
________________________________________________________________________________
Training epoch:  850
** Training err =  0.389427178997
Evaluate on Validation set:
** Validation err =  0.392573448565
________________________________________________________________________________
Training epoch:  860
** Training err =  0.389358601201
Evaluate on Validation set:
** Validation err =  0.392531419957
________________________________________________________________________________
Training epoch:  870
*

** Training err =  0.397107420744
Evaluate on Validation set:
** Validation err =  0.397484041638
________________________________________________________________________________
Training epoch:  190
** Training err =  0.396990164069
Evaluate on Validation set:
** Validation err =  0.397383219846
________________________________________________________________________________
Training epoch:  200
** Training err =  0.396881531126
Evaluate on Validation set:
** Validation err =  0.397289537597
________________________________________________________________________________
Training epoch:  210
** Training err =  0.396776608083
Evaluate on Validation set:
** Validation err =  0.397201502333
________________________________________________________________________________
Training epoch:  220
** Training err =  0.396678744321
Evaluate on Validation set:
** Validation err =  0.397117867078
________________________________________________________________________________
Training epoch:  230


** Validation err =  0.395419756641
________________________________________________________________________________
Training epoch:  600
** Training err =  0.39442941572
Evaluate on Validation set:
** Validation err =  0.395388172923
________________________________________________________________________________
Training epoch:  610
** Training err =  0.394394943248
Evaluate on Validation set:
** Validation err =  0.395365781499
________________________________________________________________________________
Training epoch:  620
** Training err =  0.394359261711
Evaluate on Validation set:
** Validation err =  0.395341495989
________________________________________________________________________________
Training epoch:  630
** Training err =  0.394311636682
Evaluate on Validation set:
** Validation err =  0.395310404555
________________________________________________________________________________
Training epoch:  640
** Training err =  0.394266526435
Evaluate on Validation set:
*

...

** Validation err =  0.398547203975
________________________________________________________________________________
Training epoch:  1040
** Training err =  0.398279238823
Evaluate on Validation set:
** Validation err =  0.398527091623
________________________________________________________________________________
Training epoch:  1050
** Training err =  0.398258495998
Evaluate on Validation set:
** Validation err =  0.398507695879
________________________________________________________________________________
Training epoch:  1060
** Training err =  0.398237703002
Evaluate on Validation set:
** Validation err =  0.398489097498
________________________________________________________________________________
Training epoch:  1070
** Training err =  0.398217261226
Evaluate on Validation set:
** Validation err =  0.39847035946
________________________________________________________________________________
Training epoch:  1080
** Training err =  0.398197583471
Evaluate on Validation s

** Validation err =  0.39789751944
________________________________________________________________________________
Training epoch:  1450
** Training err =  0.397565340907
Evaluate on Validation set:
** Validation err =  0.397884586853
________________________________________________________________________________
Training epoch:  1460
** Training err =  0.397550739848
Evaluate on Validation set:
** Validation err =  0.39787166078
________________________________________________________________________________
Training epoch:  1470
** Training err =  0.397537466887
Evaluate on Validation set:
** Validation err =  0.397861014861
________________________________________________________________________________
Training epoch:  1480
** Training err =  0.39752378688
Evaluate on Validation set:
** Validation err =  0.397849239764
________________________________________________________________________________
Training epoch:  1490
** Training err =  0.397508825259
Evaluate on Validation set

** Validation err =  0.397435625481
________________________________________________________________________________
Training epoch:  1860
** Training err =  0.397039875265
Evaluate on Validation set:
** Validation err =  0.397425981665
________________________________________________________________________________
Training epoch:  1870
** Training err =  0.397028519513
Evaluate on Validation set:
** Validation err =  0.397415947458
________________________________________________________________________________
Training epoch:  1880
** Training err =  0.397017280379
Evaluate on Validation set:
** Validation err =  0.397406218307
________________________________________________________________________________
Training epoch:  1890
** Training err =  0.397006313401
Evaluate on Validation set:
** Validation err =  0.39739683305
________________________________________________________________________________
Training epoch:  1900
** Training err =  0.396995522593
Evaluate on Validation s

** Validation err =  0.397076779626
________________________________________________________________________________
Training epoch:  2270
** Training err =  0.396620118562
Evaluate on Validation set:
** Validation err =  0.397069573836
________________________________________________________________________________
Training epoch:  2280
** Training err =  0.396610670098
Evaluate on Validation set:
** Validation err =  0.397061365157
________________________________________________________________________________
Training epoch:  2290
** Training err =  0.396601596472
Evaluate on Validation set:
** Validation err =  0.39705408692
________________________________________________________________________________
Training epoch:  2300
** Training err =  0.396592291319
Evaluate on Validation set:
** Validation err =  0.397045784049
________________________________________________________________________________
Training epoch:  2310
** Training err =  0.396583186489
Evaluate on Validation s

** Validation err =  0.396784242527
________________________________________________________________________________
Training epoch:  2680
** Training err =  0.396268273715
Evaluate on Validation set:
** Validation err =  0.396778957013
________________________________________________________________________________
Training epoch:  2690
** Training err =  0.396259316821
Evaluate on Validation set:
** Validation err =  0.396770609942
________________________________________________________________________________
Training epoch:  2700
** Training err =  0.396251555239
Evaluate on Validation set:
** Validation err =  0.396764877657
________________________________________________________________________________
Training epoch:  2710
** Training err =  0.396243432356
Evaluate on Validation set:
** Validation err =  0.396757836682
________________________________________________________________________________
Training epoch:  2720
** Training err =  0.396235568087
Evaluate on Validation 

In [9]:
grid_result

{(1e-06, 0): (999, 0.39862621077253346, 0.39838730829986924),
 (1e-05, 0): (999, 0.39450742919505438, 0.39306980029898686),
 (5e-05, 0): (870, 0.39123732468012118, 0.38637592565738316)}